In [1]:
### 540 Final Project

#### Estimation of network reliability probability by Monte Carlo Simulation 

Network reliability is a provident field by the development of the AI and the Internet, A signal sent from A to B in the network must follow a path along any available edges.

Impertinent network reliability means that the signal may fail to be transmitted correctly.

This project will focus on the computation of network failure probability ,which can evaluate the network reliability.


Method: Evaluate the probability of network failure given specific probabilities for the failure of each edge.
Assumption: Consider the simplest case: each edge is assumed to fail independently with the same probability with P.


Generate the samples:$\vec{x}=\left(x_{1}, x_{2} \ldots x_{20}\right)$,where $x_{i}'s$ ~ independent Bernouli Distribution

then the Bernouli random vector is:

$$h(\vec{x})=\left\{\begin{array}{l}0, \text { if } A \text { and } B \text { are connected (working)} \\ 1, \text { if } A \text { and } B \text { are not connected (failure) }\end{array}\right.$$


Then, the probability of network failure is:


$$E[h(\vec{x})]=\mu$$


However, Calculating the $\mu$ of any network in realistic size can be a very difficult combinational problem. 

So we introduced the `Monte Carlo simulation` to solve the problem.

the first method is Monte Carlo Integration, and the second is Impotance sampling method:


1. Monte Carlo Integration:
$\hat{u}_{M c}=\frac{1}{n} \sum_{i=1}^{n} h\left(\vec{x}_{i}\right)$

2.Impotance Sampling:
$\hat{\mu}_{I S}=\frac{1}{n} \sum_{i=1}^{n} w\left(\vec{x}_{i}^{*}\right) h\left(\vec{x}_{i}^{*}\right)$


#### 1.1 Draw the network graph

#### Assuming that the network is `undirected`, each edge has the same `weight` and `capacity`.

Since we would like to study the network failure from the beginning, we need these assuption, and the results did tell us we can have a lot work to do in the future.

The network graph we create was from the lecture 540:540 computational methods.The network faliure is related to the minimal cut of the network.So our first target is to find the minimal cut.

In [2]:
import matplotlib.pyplot as plt 
import networkx as nx 

ModuleNotFoundError: No module named 'networkx'

In [ ]:
Gn = nx.Graph()

edges_list = [('A', '1'), ('A', '2'), ('A', '3'), ('A', '4'), ('1', '2'), ('1', '5'), ('1', '6'), ('2', '6'), ('2', '3'), ('3', '7'), ('3', '4'), ('4', '7'), ('4', '8'), ('5', '6'), ('5', 'B'), ('6', '7'), ('6', 'B'), ('7', '8'), ('7', 'B'), ('8', 'B')]

pos = {'A':(-2,0), '1':(-1,1.5), '2':(-1,0.5),  '3':(-1,-0.5),  '4':(-1,-1.5),
        '5':(0,1.5), '6': (0,0.5), '7':(0,-0.5),  '8':(0,-1.5), 'B':(1,-0),}

Gn.add_edges_from(edges_list, weight=1,capacity=1) #Assuming the network is undirected, each edge has the same weight and capacity.

In [ ]:
plt.figure(figsize=(10,6))

nx.draw(Gn, pos= pos,with_labels=True,node_size=500,node_color='yellow')

# labeling the edges
nx.draw_networkx_edge_labels(Gn,pos,edge_labels={('A','1'):'$X_1$', ('A', '2'):'$X_2$',('A', '3'):"$X_3$", ('A', '4'):"$X_4$",('1', '2'):"$X_5$", ('1', '5'):"$X_8$", ('1', '6'):"$X_9$", ('2', '6'):"$X_{10}$", ('2', '3'):"$X_{6}$", ('3', '7'):"$X_{11}$", ('3', '4'):"$X_{7}$", ('4', '7'):"$X_{12}$", ('4', '8'):"$X_{13}$", ('5', '6'):"$X_{14}$", ('5', 'B'):"$X_{17}$", ('6', '7'):"$X_{15}$", ('6', 'B'):"$X_{18}$", ('7', '8'):"$X_{16}$", ('7', 'B'):"$X_{19}$", ('8', 'B'):"$X_{20}$"} ,font_size=20,font_color='red')

# plt.savefig("Evaluation of network reliability.png")

#### 1.2 Find the minimal cut of the network

In [ ]:
cut_value, partition = nx.minimum_cut(Gn,  "A", "B")
reachable, non_reachable = partition

cutset = set()
for u, nbrs in ((n, Gn[n]) for n in reachable):
    cutset.update((u, v) for v in nbrs if v in non_reachable)
print(f"The minimal cut is",sorted(cutset),"the cut set is ",partition)

# cut_value == sum(Gn.edges[u, v]["capacity"] for (u, v) in cutset)

#### So we find the minimal cut of this network, the edges are $X_{17}$ ~ $X_{20}$,which means if the four edges is not working as the same time, the network will fail, the signal will not be transmitted from A to B.Next we will calculted the failure probability.

#### 2.1 Generate the bernoulli random samples

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as st
from scipy.stats import bernoulli

In [ ]:
# generate 20-dimension bernoulli verctor
np.random.seed(seed=540)

N=100000
p = 0.1
n=20

bernoulli_rvs=bernoulli.rvs(size=N * n,p=p)
data=np.reshape(np.mat(bernoulli_rvs),(N,n))

h_x =pd.DataFrame(data)
h_x.head() # samples generated from the bernouli distribution

#### 2.2 calculate the failure probability of each edge using monte carlo integration 

In [ ]:
p_edge_fail = h_x.sum() / N #monte carlo integration
p_edge_fail #the failure probability of each edge 

The failure probability of edge $X_{17}$ ~ $X_{20}$:

In [ ]:
p_17 = p_edge_fail[16]
p_18 = p_edge_fail[17]
p_19 = p_edge_fail[18]
p_20 = p_edge_fail[19]

In [ ]:
P_fail_MCI = round(p_17 * p_18 * p_19 * p_20,7)

In [ ]:
print(f"the network failure probability is {P_fail_MCI} by the monte carlo integration method.")

we can see when the failure probability of each edge is small, the $h(\vec{x}) $  rarely reaches 1.So a large amount of simulations should be performed to estimate the u with sufficient  precision. To be more accurate , we need the IS sampling, to improve the integral approximation: use importance sampling approach.

Previously, we generate the bernouli random samples $\vec{x}=\left(x_{1}, x_{2} \ldots x_{20}\right)$. Now with the Importance samplling method, we need to generate bernouli random samples $\vec{x}_{1}^{*}, \vec{x}_{2}^{*}, \ldots, \vec{x}_{n}^{*}$ from $ g(\vec{x})=g\left(x_{1}, x_{2} \cdots, x_{20}\right)$ ,where $\left.x_{i} \text { ~Bernouli }\left(p^{*}\right) \text { where } p^{*}\right \rangle p$

$g(\vec{x})$ is the proposal function in the Importance samplling approach.

#### 2.3 generate bernouli random samples $\vec{x}_{1}^{*}, \vec{x}_{2}^{*}, \ldots, \vec{x}_{n}^{*}$ 

In [ ]:
np.random.seed(seed=42)

p_star = 0.2 #p_star should be greater than p
n=20

bernoulli_rvs_star=bernoulli.rvs(size=N * n,p=p_star)
data_star =np.reshape(np.mat(bernoulli_rvs_star),(N,n))

h_x_star =pd.DataFrame(data_star)
h_x_star.head()

In [ ]:
n_faill = h_x_star.T.sum()
n_faill.head()

In [ ]:
x_17 = h_x_star[16]
x_18 = h_x_star[17]
x_19 = h_x_star[18]
x_20 = h_x_star[19]

In [ ]:

# h_x_star.loc[(x_17 == 1) & (x_18 ==1) & (x_19 ==1) & (x_20 == 1)].head()
# h_x_star.drop('name',axis=1)
# net_faill= h_x_star[(x_17 == 1) & (x_18 ==1) & (x_19 ==1) & (x_20 == 1)]
# net_faill.head()

In [ ]:
h_x_star.loc[(x_17 == 1) & (x_18 ==1) & (x_19 ==1) & (x_20 == 1),'hx'] = 1  #label the failure network as 1

In [ ]:
h_x_star['hx'].fillna(0,inplace=True) #label the working network as 0

In [ ]:
h_x_star_plus = h_x_star['hx']  #extract the h_x

next, we calculate the importance sampling weight :

$$
\begin{array}{c}
w\left(\overrightarrow{x_{i}^{*}}\right)=\frac{f\left(\overrightarrow{x_{i}^{*}}\right)}{g\left(\overrightarrow{x_{i}^{*}}\right)}=\frac{p^{x_{i 1}^{*}}(1-p)^{1-x_{i 1}^{*}} p^{x_{i 2}^{*}}(1-p)^{1-x_{i 2}^{*}} \ldots \ldots p^{x_{i 20}^{*}}(1-p)^{1-x_{i 20}^{*}}}{p^{*} x_{i 1}^{*}\left(1-p^{*}\right)^{1-x_{i 1}^{*}} p^{*} x_{i 2}^{*}\left(1-p^{*}\right)^{1-x_{i 2}^{*}} \ldots . . p^{* x_{i 20}^{*}\left(1-p^{*}\right)^{1-x_{i 20}^{*}}}} \\
=\frac{p^{\sum_{i=1}^{i=20} x_{i j}}(1-p)^{20-\sum_{i=1}^{i=20} x_{i j}^{*}}}{p^{*} \sum_{i=1}^{i=20} x_{i j}\left(1-p^{*}\right)^{20-\sum_{i=1}^{i=20} x_{i j}^{*}}}
\end{array}
$$


In [ ]:
w_star = p ** n_faill * (1-p)**(n-n_faill) /(p_star ** n_faill * (1-p_star) ** (n-n_faill)) #importance sampling weight
pd.DataFrame(w_star).head()

In [ ]:
P_fail_IS =  round(sum(w_star * h_x_star_plus)/N ,7)

In [ ]:
print(f"the network failure probability is {P_fail_IS} by the Impotance sampling method")

The result shows us the probablity of netwok failure could be very small(on the other hand ,the reliability is very high), the bit error rate for many types signal transmission can range from $ 10 ^{-10} $ to $10^{-3}$ or even lower.